In [1]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint

In [2]:
torch.__version__

'2.1.0.post100'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cpu')

In [5]:
data = open('input.txt', 'r').read()
data[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [6]:
# for i in range(len(data)):
#     data[i] = data[i].strip("\n").lower()   
# # remove all elements in data which are empty strings
# data = list(filter(None, data))
# len(data)

In [7]:
data[:10]

'First Citi'

In [8]:
# words = []
# for name in data:
#     for word in name.split():
#         words.append(word)
# len(words)

In [9]:
# words[:10]

In [10]:
unique_chars = list(set(''.join(data)))
unique_chars.sort()
vocab_dict = {i:ch for i, ch in enumerate(unique_chars)}
vocab_dict_inv = {ch:i for i, ch in enumerate(unique_chars)}
vocab_dict, vocab_dict_inv

({0: '\n',
  1: ' ',
  2: '!',
  3: '$',
  4: '&',
  5: "'",
  6: ',',
  7: '-',
  8: '.',
  9: '3',
  10: ':',
  11: ';',
  12: '?',
  13: 'A',
  14: 'B',
  15: 'C',
  16: 'D',
  17: 'E',
  18: 'F',
  19: 'G',
  20: 'H',
  21: 'I',
  22: 'J',
  23: 'K',
  24: 'L',
  25: 'M',
  26: 'N',
  27: 'O',
  28: 'P',
  29: 'Q',
  30: 'R',
  31: 'S',
  32: 'T',
  33: 'U',
  34: 'V',
  35: 'W',
  36: 'X',
  37: 'Y',
  38: 'Z',
  39: 'a',
  40: 'b',
  41: 'c',
  42: 'd',
  43: 'e',
  44: 'f',
  45: 'g',
  46: 'h',
  47: 'i',
  48: 'j',
  49: 'k',
  50: 'l',
  51: 'm',
  52: 'n',
  53: 'o',
  54: 'p',
  55: 'q',
  56: 'r',
  57: 's',
  58: 't',
  59: 'u',
  60: 'v',
  61: 'w',
  62: 'x',
  63: 'y',
  64: 'z'},
 {'\n': 0,
  ' ': 1,
  '!': 2,
  '$': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '3': 9,
  ':': 10,
  ';': 11,
  '?': 12,
  'A': 13,
  'B': 14,
  'C': 15,
  'D': 16,
  'E': 17,
  'F': 18,
  'G': 19,
  'H': 20,
  'I': 21,
  'J': 22,
  'K': 23,
  'L': 24,
  'M': 25,
  'N': 26,
  'O'

In [11]:
# block_size = 8 # context length: how many characters do we take to predict the next one?
# X, Y = [], []
# for w in words[:]:
#   context = [0] * block_size
#   for ch in w + ' ':
#     ix = vocab_dict_inv[ch]
#     X.append(context)
#     Y.append(ix)
#     print(''.join(vocab_dict[i] for i in context), '--->', vocab_dict[ix])
#     context = context[1:] + [ix] # crop and append
# X = torch.tensor(X)
# Y = torch.tensor(Y)


In [12]:
X, Y = [], []
block_size = 8
for i in range(0, len(data)-block_size, 1):
    X.append([vocab_dict_inv[ch] for ch in data[i:i+block_size]])
    Y.append(vocab_dict_inv[data[i+block_size]])
X = torch.tensor(X)
Y = torch.tensor(Y)


In [13]:
for i in range(block_size):
    print(''.join(vocab_dict[X[0][i].item()]), end='')
print(' --->', vocab_dict[Y[0].item()])

First Ci ---> t


In [14]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([1115386, 8]), torch.int64, torch.Size([1115386]), torch.int64)

In [15]:
# Embedding layer for the context

emb_dim = 8
emb = torch.nn.Embedding(len(vocab_dict), emb_dim)

In [16]:
emb.weight

Parameter containing:
tensor([[ 6.5535e-01, -2.9290e-01,  1.9644e+00,  3.2271e-01,  1.2357e-01,
          7.7596e-02, -2.3553e+00, -1.1443e+00],
        [-1.6517e+00,  4.9936e-01,  9.9318e-03, -1.1640e+00,  1.3046e+00,
          7.2440e-01,  1.2430e+00, -6.4539e-01],
        [ 1.0337e-01, -1.0138e+00, -2.0220e-01, -1.7904e+00,  1.0182e-01,
         -3.6816e-01, -2.0775e+00,  3.7345e-01],
        [ 1.6400e-01,  1.4959e+00,  6.7536e-01,  8.7700e-01,  5.2813e-01,
         -3.3672e-01,  2.6096e-01, -9.6487e-01],
        [ 7.4527e-01,  2.1738e-01,  3.4836e-01, -9.0853e-01, -1.4168e+00,
         -7.4028e-01, -1.9314e+00, -3.4593e-01],
        [ 4.5571e-01,  1.1409e+00,  7.3276e-01,  7.9037e-01, -7.8775e-01,
          2.4492e-01,  2.0399e+00,  2.0985e-01],
        [ 7.1690e-02, -7.9993e-01, -6.3169e-01,  5.5790e-01,  2.1350e-01,
          2.8275e-01, -6.7363e-01, -1.2461e-01],
        [ 1.9392e+00,  4.6576e-01,  1.3959e+00, -8.6556e-01,  4.7334e-01,
         -3.8289e-01,  5.5716e-01,  1.4819e

In [17]:
emb.weight.shape

torch.Size([65, 8])

In [18]:
class NextChar(nn.Module):
  def __init__(self, block_size, vocab_size, emb_dim, hidden_dims = [block_size * emb_dim, block_size * emb_dim]):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, emb_dim)
    self.lin1 = nn.Linear(block_size * emb_dim, hidden_dims[0])
    self.lin2 = nn.Linear(hidden_dims[0], hidden_dims[1])
    self.lin3 = nn.Linear(hidden_dims[1], vocab_size)

  def forward(self, x):
    x = self.emb(x)
    x = x.view(x.shape[0], -1)
    x = torch.sin(self.lin1(x))
    x = torch.sin(self.lin2(x))
    x = self.lin3(x)
    return x
    

In [19]:
# Generate names from untrained model


model = NextChar(block_size, len(vocab_dict), emb_dim).to(device)
# model = torch.compile(model)

g = torch.Generator()
g.manual_seed(4000002)
def generate_name(model,sentence, itos, stoi, block_size, max_len=10):
    original_sentence = sentence
    if len(sentence) < block_size:
        sentence = " " * (block_size - len(sentence)) + sentence
    using_for_predicction = sentence[-block_size:].lower()
    context = [stoi[word] for word in using_for_predicction]
    prediction = ""
    for i in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        ch = itos[ix]
        prediction += ch
        context = context[1:] + [ix]

    return original_sentence + prediction

for i in range(10):
    want = input("Do you want to generate text? (yes/no): ")
    if want == "no":
        break
    sentence = input("Enter a sentence: ")
    print(generate_name(model,sentence, vocab_dict, vocab_dict_inv, block_size, 100))

In [21]:
for param_name, param in model.named_parameters():
    print(param_name, param.shape)

emb.weight torch.Size([65, 8])
lin1.weight torch.Size([64, 64])
lin1.bias torch.Size([64])
lin2.weight torch.Size([64, 64])
lin2.bias torch.Size([64])
lin3.weight torch.Size([65, 64])
lin3.bias torch.Size([65])


In [22]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.001)
import time
# Mini-batch training
batch_size = 4096
print_every = 100
elapsed_time = []

In [23]:
# Train the model

for epoch in range(10000):
    start_time = time.time()
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i+batch_size]
        y = Y[i:i+batch_size]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
    end_time = time.time()
    elapsed_time.append(end_time - start_time)
    if epoch % print_every == 0:
        print(epoch, loss.item())


0 1.6574249267578125


KeyboardInterrupt: 

In [27]:
for i in range(10):
    want = input("Do you want to generate text? (yes/no): ")
    if want == "no":
        break
    sentence = input("Enter a sentence: ")
    print(generate_name(model,sentence, vocab_dict, vocab_dict_inv, block_size,100))

hello my name is vannshckjour,
His Hitherd. Year.

QUEEN
ISET:
To qoer lay I wlom,
A gongether in thoud is they it is nay s


Tuning knobs

1. Embedding size
2. MLP 
3. Context length

In [25]:
path = "model.pth"

In [58]:
torch.save(model.state_dict(), path)

In [26]:
model = NextChar(block_size, len(vocab_dict), emb_dim).to(device)
model.load_state_dict(torch.load(path))

<All keys matched successfully>